In [2]:
from scipy.special import j1
from scipy.optimize import brentq
from sage.all import *
import struct

DR = RealField(53)

DD = RealField(157)

def double_to_hex(f):
    # Converts Python float (f64) to hex string
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def format_dyadic_hex(value):
    l = hex(value)[2:]
    n = 8
    x = [l[i:i + n] for i in range(0, len(l), n)]
    return "0x" + "_".join(x) + "_u128"

def print_dyadic(value):
    (s, m, e) = RealField(128)(value).sign_mantissa_exponent();
    print("DyadicFloat128 {")
    print(f"    sign: DyadicSign::{'Pos' if s >= 0 else 'Neg'},")
    print(f"    exponent: {e},")
    print(f"    mantissa: {format_dyadic_hex(m)},")
    print("},")


In [6]:
def build_sollya_script(idx):
    return f"""
d = [{idx}/8, {idx + 1}/8];

f = x/erf(x);
Q = fpminimax(f, [|0, 2, 4, 6, 8, 10, 12, 14|], [|D...|], d);

for i from 0 to degree(Q) by 2 do {{
    write(coeff(Q, i)) >> "coefficients.txt";
    write("\\n") >> "coefficients.txt";
}};
"""

def load_coefficients(filename):
    with open(filename, "r") as f:
        return [RealField(500)(line.strip()) for line in f if line.strip()]

def call_sollya_on_interval(idx):
    sollya_script = build_sollya_script(idx)
    with open("tmp_interval.sollya", "w") as f:
        f.write(sollya_script)
    import subprocess
    if os.path.exists("coefficients.txt"):
        os.remove("coefficients.txt")
    try:
        result = subprocess.run(
            ["sollya", "tmp_interval.sollya"],
            check=True,
            capture_output=True,
            text=True
        )
    except subprocess.CalledProcessError as e:
        return

def print_coeffs(poly):
    print("[")
    for i in range(len(poly)):
        coeff = poly[i]
        print(double_to_hex(coeff), ",")
    print("],")

print(f"static COEFFS: [[u64; 8]; 32] = [")
for i in range(0, 32):
    call_sollya_on_interval(i)
    coeffs = load_coefficients(f"coefficients.txt")
    print_coeffs(coeffs)
print("];")

static COEFFS: [[u64; 8]; 32] = [
[
0x3fec5bf891b4ef6b ,
0x3fd2e7fb0bcdee7f ,
0x3f842aa5641a200a ,
0xbf752081ae81d16e ,
0x3f2e1a191fb85592 ,
0xbf203a20ad500043 ,
0x3f861a864f719e76 ,
0xbfc79f68bad20bd1 ,
],
[
0x3fec5bf891b4ef6b ,
0x3fd2e7fb0bcdf45b ,
0x3f842aa561f35512 ,
0xbf75207c8167ac1d ,
0x3f2db4b119b4ce20 ,
0x3f20fa28737c4219 ,
0xbef38e74cca2219a ,
0xbec5d70713fc621e ,
],
[
0x3fec5bf891b4ef30 ,
0x3fd2e7fb0bce1c0f ,
0x3f842aa56138541f ,
0xbf75207c6197eb7c ,
0x3f2db4799120e074 ,
0x3f20fc28d915a6e9 ,
0xbef3ea5b479dc053 ,
0xbebbffe6df8ec372 ,
],
[
0x3fec5bf891b4bf18 ,
0x3fd2e7fb0bde166f ,
0x3f842aa53c721766 ,
0xbf7520796733bbec ,
0x3f2db21eebf4144f ,
0x3f210545cc78d0f0 ,
0xbef48ad7e4aa2d10 ,
0xbeb24a043ad31907 ,
],
[
0x3fec5bf891ab16e9 ,
0x3fd2e7fb0dc7b919 ,
0x3f842aa29d8381e7 ,
0xbf7520592585d601 ,
0x3f2da30df1566e43 ,
0x3f212780ff325aa6 ,
0xbef5e98ea9819e42 ,
0xbe9849d52099dcb9 ,
],
[
0x3fec5bf890ddfa8d ,
0x3fd2e7fb28aab312 ,
0x3f842a8a461f0eb7 ,
0xbf751f93b2d27114 ,
0x3f2d66789eed5